Extra package install (install in conda env)

# Load data file
Read headlines file and add to variable:content

In [76]:
import os
pwd = os.getcwd()
path = os.path.join(pwd, "data", "headlines.txt")

f = open(path,"r")
lines = f.readlines()
f.close()




# Build regular expression from cities

In [77]:
import re
import geonamescache
from unidecode import unidecode

def byLength(e):
    return len(e)

gc = geonamescache.GeonamesCache()
cityNames = []
for city in gc.get_cities().values():
        name = city['name']
        normalizedName = unidecode(city['name'])
        cityNames.append(normalizedName)

#sort from long to short (want to match on long names first)
cityNames.sort(reverse=True, key=byLength)

orCities = "|".join(cityNames)
regex = "\\b(" + orCities + ")\\b"
compiled_re = re.compile(regex)

# Add headline, city and country code to panda data frame

In [112]:
import re
import geonamescache
from unidecode import unidecode
import pandas as pd 

df = pd.DataFrame(columns =['headline', 'city', 'country']) 

for line in lines:
    m = compiled_re.search(line)
    if m != None:
        cityName = line[m.start():m.end()]
        cities = gc.get_cities_by_name(cityName)
        if len(cities)==0:
            df = df.append({'headline' : line, "city" : "" , "country" : ""} , ignore_index=True)
        else:
            
            largestCityAttr = {}
            for city in cities:
                #there should only be one
                for cityKey in city.keys():
                    cityAttr = city[cityKey]
                    if len(largestCityAttr)==0:
                        largestCityAttr=cityAttr
                    else:
                        if cityAttr["population"] > largestCityAttr["population"]:
                            largestCityAttr = cityAttr
            df = df.append({'headline' : line, "city" : largestCityAttr["name"] , "country" : largestCityAttr["countrycode"]} , ignore_index=True)
    else:
        df = df.append({'headline' : line, "city" : "" , "country" : ""} , ignore_index=True)
    
print(df)

pwd = os.getcwd()
path = os.path.join(pwd, "data", "cities2headlines.csv")

df.to_csv(path, index=False)

                                              headline           city country
0                           Zika Outbreak Hits Miami\n          Miami      US
1                    Could Zika Reach New York City?\n  New York City      US
2                  First Case of Zika in Miami Beach\n    Miami Beach      US
3            Mystery Virus Spreads in Recife, Brazil\n         Recife      BR
4            Dallas man comes down with case of Zika\n         Dallas      US
..                                                 ...            ...     ...
645  Rumors about Rabies spreading in Jerusalem hav...      Jerusalem      IL
646            More Zika patients reported in Indang\n         Indang      PH
647  Suva authorities confirmed the spread of Rotav...           Suva      FJ
648       More Zika patients reported in Bella Vista\n    Bella Vista      DO
649                   Zika Outbreak in Wichita Falls\n  Wichita Falls      US

[650 rows x 3 columns]


# Lookup Lat and Long, use city with largest population

In [113]:
def findLat(cityName):
   
    if cityName=="":
        return [0,0,""]
    cities = gc.get_cities_by_name(cityName)
    if len(cities)==0:
        return [0,0,""]
    else:
        largestCityAttr = {}
        for city in cities:
            #each city is a dict with a key as id and value is city properties??
            for cityKey in city.keys():
                cityAttr = city[cityKey]
                if len(largestCityAttr)==0:
                    largestCityAttr=cityAttr
                else:
                    if cityAttr["population"] > largestCityAttr["population"]:
                        largestCityAttr = cityAttr
        return [largestCityAttr["latitude"],largestCityAttr["longitude"],largestCityAttr["countrycode"]]
    


print(df)
df["info"]= df["city"].apply(lambda x : findLat(x))
df["lat"]=df["info"].apply(lambda x : x[0])
df["long"]=df["info"].apply(lambda x : x[1])
df["country"]=df["info"].apply(lambda x : x[2])
df.drop(columns=["info"])
print(df)


                                              headline           city country
0                           Zika Outbreak Hits Miami\n          Miami      US
1                    Could Zika Reach New York City?\n  New York City      US
2                  First Case of Zika in Miami Beach\n    Miami Beach      US
3            Mystery Virus Spreads in Recife, Brazil\n         Recife      BR
4            Dallas man comes down with case of Zika\n         Dallas      US
..                                                 ...            ...     ...
645  Rumors about Rabies spreading in Jerusalem hav...      Jerusalem      IL
646            More Zika patients reported in Indang\n         Indang      PH
647  Suva authorities confirmed the spread of Rotav...           Suva      FJ
648       More Zika patients reported in Bella Vista\n    Bella Vista      DO
649                   Zika Outbreak in Wichita Falls\n  Wichita Falls      US

[650 rows x 3 columns]
                                        